# Block 1.2 & 1.3: Python Grundlagen und Losgrößenoptimierung
### Intelligente Produktionsplanung und KI
Universität Stuttgart

---


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# SciPy optional (wird hier nicht benötigt, Import aber tolerant gehalten)
try:
    from scipy.optimize import minimize_scalar
except Exception:
    minimize_scalar = None

warnings.filterwarnings('ignore')

# Konfiguration für deutsche Plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("🎯 Willkommen zur Losgrößenoptimierung!")
print("=" * 50)


## 1. GRUNDLAGEN: EOQ-MODELL (Economic Order Quantity)
Das EOQ-Modell minimiert Gesamtkosten aus Bestell- und Lagerkosten durch Wahl der optimalen Losgröße Q.


In [ ]:
def eoq_kosten(Q, D, K, h):
    """
    Berechnet die Gesamtkosten für eine gegebene Losgröße Q
    
    Parameter:
    Q: Losgröße (Stück)
    D: Jahresbedarf (Stück/Jahr)
    K: Bestellkosten pro Bestellung (€)
    h: Lagerhaltungskosten pro Stück und Jahr (€/Stück/Jahr)
    """
    if Q <= 0:
        return float('inf')
    
    bestellkosten = (D / Q) * K
    lagerkosten = (Q / 2) * h
    gesamtkosten = bestellkosten + lagerkosten
    
    return gesamtkosten

def eoq_optimal(D, K, h):
    """Berechnet die optimale Losgröße nach EOQ-Formel"""
    Q_opt = np.sqrt((2 * D * K) / h)
    kosten_opt = eoq_kosten(Q_opt, D, K, h)
    return Q_opt, kosten_opt


## 2. FALLBEISPIEL: AUTOMOBILZULIEFERER


In [ ]:
print("\n📊 FALLBEISPIEL: Automobilzulieferer GmbH")
print("-" * 40)

# Produktdaten
produkte = {
    'Bremssattel': {'D': 12000, 'K': 150, 'h': 8.50},
    'Stoßdämpfer': {'D': 8000,  'K': 200, 'h': 12.00},
    'Kupplungsscheibe': {'D': 15000, 'K': 120, 'h': 6.80}
}

ergebnisse = {}

for produkt, daten in produkte.items():
    Q_opt, kosten_opt = eoq_optimal(daten['D'], daten['K'], daten['h'])
    ergebnisse[produkt] = {
        'Q_optimal': Q_opt,
        'Kosten_optimal': kosten_opt,
        'Bestellhäufigkeit': daten['D'] / Q_opt,
        'Reichweite_Tage': (Q_opt / daten['D']) * 365
    }
    
    print(f"\n{produkt}:")
    print(f"  Optimale Losgröße: {Q_opt:.0f} Stück")
    print(f"  Minimale Kosten: {kosten_opt:.2f} €/Jahr")
    print(f"  Bestellhäufigkeit: {daten['D']/Q_opt:.1f} mal/Jahr")
    print(f"  Reichweite: {(Q_opt/daten['D'])*365:.0f} Tage")


## 3. VISUALISIERUNG DER KOSTENFUNKTION
Gesamtkosten als Summe aus Bestell- und Lagerkosten in Abhängigkeit von Q.


In [ ]:
def plot_kostenfunktion(produkt_name, D, K, h):
    """Visualisiert die Kostenfunktion für ein Produkt"""
    Q_opt, kosten_opt = eoq_optimal(D, K, h)
    
    # Bereich für Q-Werte
    Q_range = np.linspace(50, Q_opt * 3, 1000)
    
    # Kostenkomponenten berechnen
    bestellkosten = [(D / Q) * K for Q in Q_range]
    lagerkosten = [(Q / 2) * h for Q in Q_range]
    gesamtkosten = [eoq_kosten(Q, D, K, h) for Q in Q_range]
    
    # Plot erstellen
    plt.figure(figsize=(12, 8))
    
    plt.plot(Q_range, bestellkosten, 'r--', label='Bestellkosten', linewidth=2)
    plt.plot(Q_range, lagerkosten,  'b--', label='Lagerkosten',  linewidth=2)
    plt.plot(Q_range, gesamtkosten, 'g-',  label='Gesamtkosten', linewidth=3)
    
    # Optimum markieren
    plt.axvline(x=Q_opt, color='orange', linestyle=':', linewidth=2, 
                label=f'Optimum: Q* = {Q_opt:.0f}')
    plt.plot(Q_opt, kosten_opt, 'ro', markersize=10, label=f'Min. Kosten: {kosten_opt:.2f} €')
    
    plt.xlabel('Losgröße Q (Stück)')
    plt.ylabel('Kosten (€/Jahr)')
    plt.title(f'EOQ-Analyse: {produkt_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Beispiel-Plot für Bremssattel
plot_kostenfunktion('Bremssattel', 
                   produkte['Bremssattel']['D'],
                   produkte['Bremssattel']['K'],
                   produkte['Bremssattel']['h'])


## 4. SENSITIVITÄTSANALYSE
Einfluss von K, h oder D auf optimale Losgröße und minimale Kosten.


In [ ]:
print("\n🔍 SENSITIVITÄTSANALYSE")
print("-" * 30)

def sensitivitaetsanalyse(D, K, h, parameter='K', variation_prozent=20):
    """
    Führt eine Sensitivitätsanalyse für einen Parameter durch
    """
    Q_basis, kosten_basis = eoq_optimal(D, K, h)
    
    # Variationen des Parameters
    if parameter == 'K':
        variationen = np.linspace(K * (1 - variation_prozent/100), 
                                  K * (1 + variation_prozent/100), 21)
        Q_variationen = [eoq_optimal(D, k, h)[0] for k in variationen]
        kosten_variationen = [eoq_optimal(D, k, h)[1] for k in variationen]
        x_label = 'Bestellkosten K (€)'
        
    elif parameter == 'h':
        variationen = np.linspace(h * (1 - variation_prozent/100), 
                                  h * (1 + variation_prozent/100), 21)
        Q_variationen = [eoq_optimal(D, K, h_var)[0] for h_var in variationen]
        kosten_variationen = [eoq_optimal(D, K, h_var)[1] for h_var in variationen]
        x_label = 'Lagerkosten h (€/Stück/Jahr)'
        
    elif parameter == 'D':
        variationen = np.linspace(D * (1 - variation_prozent/100), 
                                  D * (1 + variation_prozent/100), 21)
        Q_variationen = [eoq_optimal(d, K, h)[0] for d in variationen]
        kosten_variationen = [eoq_optimal(d, K, h)[1] for d in variationen]
        x_label = 'Jahresbedarf D (Stück/Jahr)'
    
    # Plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Optimale Losgröße
    ax1.plot(variationen, Q_variationen, 'b-', linewidth=2)
    ax1.axhline(y=Q_basis, color='r', linestyle='--', alpha=0.7, label=f'Basis: {Q_basis:.0f}')
    ax1.set_xlabel(x_label)
    ax1.set_ylabel('Optimale Losgröße Q* (Stück)')
    ax1.set_title('Einfluss auf optimale Losgröße')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Minimale Kosten
    ax2.plot(variationen, kosten_variationen, 'g-', linewidth=2)
    ax2.axhline(y=kosten_basis, color='r', linestyle='--', alpha=0.7, label=f'Basis: {kosten_basis:.2f} €')
    ax2.set_xlabel(x_label)
    ax2.set_ylabel('Minimale Kosten (€/Jahr)')
    ax2.set_title('Einfluss auf minimale Kosten')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

# Sensitivitätsanalyse für Bremssattel
print("Sensitivitätsanalyse für Bremssattel - Variation der Bestellkosten:")
sensitivitaetsanalyse(produkte['Bremssattel']['D'],
                     produkte['Bremssattel']['K'],
                     produkte['Bremssattel']['h'],
                     parameter='K')


## 5. Interaktive Aufgaben für Studierende

**Aufgabe 1: Neues Produkt analysieren**
- Jahresbedarf: 18.000 Stück
- Bestellkosten: 180 € pro Bestellung
- Lagerkosten: 9,50 € pro Stück und Jahr

Berechnen Sie:
a) Die optimale Losgröße  
b) Die minimalen Gesamtkosten  
c) Die Bestellhäufigkeit pro Jahr

_Lösung (für Dozierende, Richtwerte):_  
Q* ≈ 784 Stück; Kosten ≈ 7.448 €/Jahr; Häufigkeit ≈ 23/Jahr

**Aufgabe 2: Was-wäre-wenn Analyse (Bremssattel)**
Wie ändern sich die optimale Losgröße und die Kosten, wenn:  
a) Die Bestellkosten um 50% steigen?  
b) Die Lagerkosten um 30% sinken?  
c) Der Jahresbedarf um 25% steigt?  

Nutzen Sie die oben definierten Funktionen (eoq_optimal, eoq_kosten).


## 6. Erweiterungen des EOQ-Modells: Mengenrabatte


In [ ]:
def eoq_mit_mengenrabatt(D, K, h, rabatt_stufen):
    """
    EOQ-Modell mit Mengenrabatten
    
    rabatt_stufen: Liste von Tupeln (Mindestmenge, Preis_pro_Stück)
    """
    beste_option = None
    min_gesamtkosten = float('inf')
    
    for mindestmenge, preis in rabatt_stufen:
        # Optimale Losgröße für diesen Preis berechnen
        h_effektiv = h + preis * 0.1  # Annahme: 10% Zinssatz auf Warenwert
        Q_opt = np.sqrt((2 * D * K) / h_effektiv)
        
        # Losgröße muss mindestens die Rabattgrenze erreichen
        Q_effektiv = max(Q_opt, mindestmenge)
        
        # Gesamtkosten berechnen (inkl. Warenwert)
        bestellkosten = (D / Q_effektiv) * K
        lagerkosten = (Q_effektiv / 2) * h_effektiv
        warenkosten = D * preis
        gesamtkosten = bestellkosten + lagerkosten + warenkosten
        
        if gesamtkosten < min_gesamtkosten:
            min_gesamtkosten = gesamtkosten
            beste_option = {
                'Mindestmenge': mindestmenge,
                'Preis': preis,
                'Q_optimal': Q_opt,
                'Q_effektiv': Q_effektiv,
                'Gesamtkosten': gesamtkosten
            }
    
    return beste_option

print("\n💰 ERWEITERUNG: EOQ mit Mengenrabatten")
print("-" * 45)

# Beispiel Mengenrabatte für Bremssattel
rabatte = [
    (0,    25.00),   # Normalpreis
    (500,  24.50),   # 2% Rabatt ab 500 Stück
    (1000, 24.00),   # 4% Rabatt ab 1000 Stück
    (2000, 23.50)    # 6% Rabatt ab 2000 Stück
]

beste_rabatt_option = eoq_mit_mengenrabatt(
    produkte['Bremssattel']['D'],
    produkte['Bremssattel']['K'],
    produkte['Bremssattel']['h'],
    rabatte
)

print("Beste Option mit Mengenrabatten:")
for key, value in beste_rabatt_option.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")


## Zusammenfassung


In [ ]:
print("\n🎯 ZUSAMMENFASSUNG")
print("=" * 20)
print("✅ EOQ-Grundmodell verstanden")
print("✅ Python-Implementierung erstellt") 
print("✅ Sensitivitätsanalyse durchgeführt")
print("✅ Erweiterungen kennengelernt")
print("\n📚 Nächster Schritt: Lineare Optimierung in Block 2!")
